## Tarea 2 - Aprendizaje Automático
Vicente Mieres

In [4]:
# importacion de librerias

import tensorflow as tf
import numpy as np
# from tensorflow.keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os


# Parte 1 - Lectura de Imágenes

In [ ]:
def load_image_paths_and_labels(file_path):
    """Carga las rutas de las imágenes y las etiquetas desde un archivo de texto."""
    image_paths = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file.readlines():
            path, label = line.strip().split('\t')  
            image_paths.append(path)
            labels.append(int(label)) 
    return image_paths, np.array(labels)

def load_images(image_paths, folder_route, dataset):
    """Carga las imágenes y las aplana a vectores."""
    images = []
    if dataset == 'Animals':
        route = './QuickDraw-Animals/QuickDraw-Animals'

        


    for path in image_paths:
        with Image.open(folder_route + path) as img:
            img_array = np.array(img).reshape(-1)
            images.append(img_array)
    return np.array(images)

QuickDraw-10

In [41]:
# Cargar las rutas de entrenamiento y prueba junto con las etiquetas
train_image_paths_10, train_labels_10 = load_image_paths_and_labels("./QuickDraw-10-Tarea2/QuickDraw-10/train.txt")
test_image_paths_10, test_labels_10 = load_image_paths_and_labels("./QuickDraw-10-Tarea2/QuickDraw-10/test.txt")

# Cargar y procesar las imágenes
train_images_10 = load_images(train_image_paths_10, "QuickDraw-10-Tarea2/QuickDraw-10/")
test_images_10 = load_images(test_image_paths_10, "QuickDraw-10-Tarea2/QuickDraw-10/")

QuickDraw-Animals

In [ ]:
def load_images_and_labels(dataset):
    """
    Carga imágenes y etiquetas para train y test del dataset QuickDraw-Animals.
    
    Retorna: 
      X_test, y_test, X_train, y_train (numpy arrays)
    """
    if dataset != "Animals":
        raise ValueError("Sólo soporta dataset 'Animals'.")

    base_path = './QuickDraw-Animals/QuickDraw-Animals'
    mapping_file = os.path.join(base_path, 'mapping.txt')

    # Leer mapping.txt y crear diccionario etiqueta->número
    label_map = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                label = parts[0]
                idx = int(parts[1])
                label_map[label] = idx

    def load_images_from_folder(folder_path):
        images = []
        labels = []
        # Las subcarpetas son las clases
        for label_name in sorted(os.listdir(folder_path)):
            label_folder = os.path.join(folder_path, label_name)
            if os.path.isdir(label_folder) and label_name in label_map:
                for img_file in sorted(os.listdir(label_folder)):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(label_folder, img_file)
                        with Image.open(img_path) as img:
                            img_array = np.array(img).reshape(-1)
                            images.append(img_array)
                            labels.append(label_map[label_name])
        return np.array(images), np.array(labels)

    # Cargar test
    X_test, y_test = load_images_from_folder('./QuickDraw-Animals/QuickDraw-Animals/test_images/test_images')

    # Cargar train
    X_train, y_train = load_images_from_folder('./QuickDraw-Animals/QuickDraw-Animals/train_images/train_images')

    return X_test, y_test, X_train, y_train


In [ ]:
test_images_animals, test_labels_animals, train_images_animals, train_labels_animals = load_images_and_labels("Animals")

## Parte 2 - Construccion de modelos